In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_hub as tfhub
tf.keras.mixed_precision.set_global_policy('mixed_float16')


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3090, compute capability 8.6


In [2]:
training_set='F:/Birds/train'
test_set='F:/Birds/test'
validation_set= 'F:/Birds/valid'

img_size = 224
input_shape=(224,224,3)

In [3]:
data_aug = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(scale=1./255),
    tf.keras.layers.experimental.preprocessing.RandomFlip( mode='horizontal' ),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.4),
   # tf.keras.layers.experimental.preprocessing.RandomWidth(0.3),
   # tf.keras.layers.experimental.preprocessing.RandomHeight(0.3),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.3)

])
data_aug_test = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(scale=1./255)
])

In [4]:
train = tf.keras.preprocessing.image_dataset_from_directory(training_set, labels='inferred', label_mode='int',color_mode='rgb',batch_size=64, image_size=(img_size , img_size),shuffle=True)
num_classes = len(train.class_names)
train= train.map(lambda x,y:(data_aug(x,training=True),y),num_parallel_calls=12).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

Found 39364 files belonging to 275 classes.


In [5]:
validation = tf.keras.preprocessing.image_dataset_from_directory(validation_set,label_mode='int',color_mode='rgb',batch_size=64,image_size=(img_size,img_size),shuffle=True,seed=52)
validation = validation.map(lambda x,y:(data_aug_test(x,training=True),y),num_parallel_calls=12).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

Found 1375 files belonging to 275 classes.


In [6]:
test = tf.keras.preprocessing.image_dataset_from_directory(test_set,label_mode='int',color_mode='rgb',batch_size=128,image_size=(img_size,img_size),shuffle=True,seed=52)
test = test.map(lambda x,y:(data_aug_test(x,training=True),y),num_parallel_calls=12).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

Found 1375 files belonging to 275 classes.


In [7]:
model_path = 'F:/Birds/checkpoint'
callback = tf.keras.callbacks.ModelCheckpoint(model_path, save_weights_only=True, monitor='val_acc',save_best_only=True)

In [8]:
transfer_model = tf.keras.applications.Xception(include_top=False)
transfer_model.trainable=False
#tf.keras.utils.plot_model(transfer_model,show_shapes=True,expand_nested=True)

83689472/83683744 [==============================] - 2s 0us/step


In [9]:
inputs = tf.keras.layers.Input(shape=input_shape, name='inputs')

In [10]:
x = transfer_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D(name='pooling')(x)
x = tf.keras.layers.Dense(num_classes)(x)
outputs = tf.keras.layers.Activation('softmax',dtype=tf.float32,name='softmax_folat32')(x)

In [11]:
model = tf.keras.Model(inputs,outputs)

In [12]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer = 'adam',
              metrics=['acc']
              )

In [13]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 224, 224, 3)]     0         
_________________________________________________________________
xception (Functional)        (None, None, None, 2048)  20861480  
_________________________________________________________________
pooling (GlobalAveragePoolin (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 275)               563475    
_________________________________________________________________
softmax_folat32 (Activation) (None, 275)               0         
Total params: 21,424,955
Trainable params: 563,475
Non-trainable params: 20,861,480
_________________________________________________________________


In [14]:
for layer in model.layers:
    print(layer.name, layer.dtype, layer.dtype_policy)

inputs float32 <Policy "float32">
xception float32 <Policy "mixed_float16">
pooling float32 <Policy "mixed_float16">
dense float32 <Policy "mixed_float16">
softmax_folat32 float32 <Policy "float32">


In [15]:
future_extracted_model = model.fit(train,epochs=5,validation_data=validation,callbacks=[callback])

Epoch 1/5
616/616 [==============================] - 62s 94ms/step - loss: 2.3030 - acc: 0.5085 - val_loss: 0.8855 - val_acc: 0.8044
Epoch 2/5
616/616 [==============================] - 22s 35ms/step - loss: 1.1087 - acc: 0.7330 - val_loss: 0.6280 - val_acc: 0.8596
Epoch 3/5
616/616 [==============================] - 22s 35ms/step - loss: 0.8196 - acc: 0.8056 - val_loss: 0.5325 - val_acc: 0.8800
Epoch 4/5
616/616 [==============================] - 21s 35ms/step - loss: 0.6427 - acc: 0.8513 - val_loss: 0.4818 - val_acc: 0.8858
Epoch 5/5
616/616 [==============================] - 21s 35ms/step - loss: 0.5161 - acc: 0.8872 - val_loss: 0.4511 - val_acc: 0.8873


In [16]:
model.evaluate(test)

11/11 [==============================] - 1s 62ms/step - loss: 0.4042 - acc: 0.9062


[0.40416011214256287, 0.906181812286377]

In [17]:
transfer_model.trainable=True

for layer in transfer_model.layers[:-20]:
    layer.trainable=False

In [18]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics=['acc']
)

In [19]:
for layer in model.layers:
    print (layer.name, layer.trainable,layer.dtype_policy)

inputs True <Policy "float32">
xception True <Policy "mixed_float16">
pooling True <Policy "mixed_float16">
dense True <Policy "mixed_float16">
softmax_folat32 True <Policy "float32">


In [20]:
for layer_number, layer in enumerate(transfer_model.layers):
    print (layer_number, layer.name, layer.trainable)

0 input_1 False
1 block1_conv1 False
2 block1_conv1_bn False
3 block1_conv1_act False
4 block1_conv2 False
5 block1_conv2_bn False
6 block1_conv2_act False
7 block2_sepconv1 False
8 block2_sepconv1_bn False
9 block2_sepconv2_act False
10 block2_sepconv2 False
11 block2_sepconv2_bn False
12 conv2d False
13 block2_pool False
14 batch_normalization False
15 add False
16 block3_sepconv1_act False
17 block3_sepconv1 False
18 block3_sepconv1_bn False
19 block3_sepconv2_act False
20 block3_sepconv2 False
21 block3_sepconv2_bn False
22 conv2d_1 False
23 block3_pool False
24 batch_normalization_1 False
25 add_1 False
26 block4_sepconv1_act False
27 block4_sepconv1 False
28 block4_sepconv1_bn False
29 block4_sepconv2_act False
30 block4_sepconv2 False
31 block4_sepconv2_bn False
32 conv2d_2 False
33 block4_pool False
34 batch_normalization_2 False
35 add_2 False
36 block5_sepconv1_act False
37 block5_sepconv1 False
38 block5_sepconv1_bn False
39 block5_sepconv2_act False
40 block5_sepconv2 False

In [21]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 224, 224, 3)]     0         
_________________________________________________________________
xception (Functional)        (None, None, None, 2048)  20861480  
_________________________________________________________________
pooling (GlobalAveragePoolin (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 275)               563475    
_________________________________________________________________
softmax_folat32 (Activation) (None, 275)               0         
Total params: 21,424,955
Trainable params: 7,889,851
Non-trainable params: 13,535,104
_________________________________________________________________


In [22]:
epochs_2 = 10
fine_tuned_model = model.fit(
    train,
    epochs=epochs_2,
    validation_data=validation,
    initial_epoch=future_extracted_model.epoch[-1]
)

Epoch 5/10
616/616 [==============================] - 29s 44ms/step - loss: 0.2980 - acc: 0.9171 - val_loss: 0.2384 - val_acc: 0.9316
Epoch 6/10
616/616 [==============================] - 27s 44ms/step - loss: 0.0952 - acc: 0.9749 - val_loss: 0.2467 - val_acc: 0.9353
Epoch 7/10
616/616 [==============================] - 28s 46ms/step - loss: 0.0428 - acc: 0.9890 - val_loss: 0.2439 - val_acc: 0.9396
Epoch 8/10
616/616 [==============================] - 28s 45ms/step - loss: 0.0348 - acc: 0.9899 - val_loss: 0.3002 - val_acc: 0.9229
Epoch 9/10
616/616 [==============================] - 27s 44ms/step - loss: 0.0313 - acc: 0.9910 - val_loss: 0.2775 - val_acc: 0.9324
Epoch 10/10
616/616 [==============================] - 28s 45ms/step - loss: 0.0246 - acc: 0.9929 - val_loss: 0.2786 - val_acc: 0.9433


In [23]:
for i in range(5):
    model.evaluate(test)

11/11 [==============================] - 1s 53ms/step - loss: 0.1515 - acc: 0.9527


In [ ]:
model.save('fine_tuned_model.h5')

In [ ]:
saved_model2 =  tf.keras.models.load_model('fine_tuned_model.h5')

In [ ]:
saved_model2.evaluate(test)

In [30]:
#image = 'F:/Birds/images_to_test/3.jpg'
image = 'C:/Users/akghost96/Downloads/images.jpg'
img = tf.keras.preprocessing.image.load_img(image, target_size=(img_size,img_size))
img = tf.keras.preprocessing.image.img_to_array(img)
img = tf.expand_dims(img,0)
img = img/255

model.predict(img)

array([[9.99626160e-01, 7.60257969e-16, 9.49009009e-22, 5.98890055e-18,
        2.71078310e-16, 7.00173399e-18, 6.57752020e-18, 1.32312050e-19,
        8.99040439e-18, 1.39246095e-17, 5.81338536e-21, 1.36101701e-13,
        2.22514629e-17, 2.00842745e-10, 1.16589334e-16, 1.55291957e-06,
        9.09366499e-20, 8.27987566e-20, 9.77661103e-19, 7.51625053e-14,
        5.99792216e-21, 5.10717900e-12, 1.98623888e-19, 6.38475471e-21,
        1.54223436e-13, 1.26976005e-20, 2.11116001e-16, 6.41931924e-11,
        1.70147945e-22, 5.11487325e-15, 2.22849824e-20, 4.99185155e-08,
        2.97273491e-13, 8.80062534e-17, 3.03227200e-11, 9.24790713e-12,
        9.36821616e-17, 1.34961945e-17, 1.54455770e-16, 6.76591292e-12,
        1.01874621e-17, 7.00173399e-18, 7.23262505e-10, 1.15403179e-06,
        5.85366242e-14, 3.30738469e-18, 1.70147945e-22, 1.19103495e-17,
        3.66864627e-17, 2.09033173e-17, 5.46117040e-21, 4.85285606e-14,
        7.61403262e-19, 2.01992867e-11, 1.03044713e-19, 2.026018